# Exercises for Building a Recommendation Engine

# Systems check: imports and files

In [1]:
import numpy as np
import pandas as pd

# Pandas questions: Series and DataFrames
## 1. Adding a column in a DataFrame

In [2]:
# given the following DataFrame, add a new column to it
df = pd.DataFrame({'col1': [1,2,3,4]})
df['new_column'] = [5,6,7,8]
df

,col1,new_column
0,1,5
1,2,6
2,3,7
3,4,8


## 2. Deleting a row in a DataFrame

In [3]:
# given the following DataFrame, delete row 'd' from it
df = pd.DataFrame({'col1': [1,2,3,4]}, index = ['a','b','c','d'])
df.drop('d')

,col1
a,1
b,2
c,3


## 3. Creating a DataFrame from a few Series

In [4]:
# given the following three Series, create a DataFrame such that it holds them as its columns
ser_1 = pd.Series(np.random.randn(6))
ser_2 = pd.Series(np.random.randn(6))
ser_3 = pd.Series(np.random.randn(6))
pd.DataFrame({'col_1':ser_1, 'col_2':ser_2, 'col_3':ser_3})

,col_1,col_2,col_3
0,-0.945558,-0.056763,-1.165976
1,-0.017926,-0.834824,-0.781241
2,-0.489358,-0.497496,-0.396399
3,-1.178609,-0.046132,-0.972187
4,-0.568267,-0.585932,-0.198764
5,0.362769,0.644332,-0.904400


# Pandas questions: Indexing

## 1. Indexing into a specific column

In [5]:
# given the following DataFrame, try to index into the 'col_2' column
df = pd.DataFrame(data={'col_1': [0.12, 7, 45, 10], 'col_2': [0.9, 9, 34, 11]},
                  columns=['col_1', 'col_2', 'col_3'],
                  index=['obs1', 'obs2', 'obs3', 'obs4'])
df['col_2']

obs1     0.9
obs2     9.0
obs3    34.0
obs4    11.0
Name: col_2, dtype: float64

## 2. Label-based indexing

In [6]:
# using the same DataFrame, index into the row whose index is 'obs3'
df.ix[2]

col_1     45
col_2     34
col_3    NaN
Name: obs3, dtype: object

## 2. Position-based indexing

In [7]:
# using the same DataFrame, index into into its first row
df.iloc[0]

col_1    0.12
col_2     0.9
col_3     NaN
Name: obs1, dtype: object

# Mini-Challenge prep: data loading

## 1. How to load the `users` and `movies` portions of MovieLens

In [8]:
import pandas as pd

users = pd.read_table('data/ml-1m/users.dat',
                      sep='::', header=None, engine='python',
                      names=['user_id', 'gender', 'age', 'occupation', 'zip'])

movies = pd.read_table('data/ml-1m/movies.dat',
                       sep='::', header=None, engine='python',
                       names=['movie_id', 'title', 'genres'])

## 2. How to load the training and testing subsets

In [9]:
# subset version (hosted notebook)
movielens_train = pd.read_csv('data/movielens_train.csv', index_col=0, encoding='latin3')
movielens_test = pd.read_csv('data/movielens_test.csv', index_col=0, encoding='latin3')

In [10]:
movielens_train.head()

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres,for_testing
593263,3562,3798,4,967332344,F,25,6,32812,What Lies Beneath (2000),Thriller,False
235597,1051,3793,4,974958593,F,25,0,60513,X-Men (2000),Action|Sci-Fi,False
219003,3727,2366,3,966309522,M,35,7,74401,King Kong (1933),Action|Adventure|Horror,False
685090,4666,1094,3,963843918,M,35,1,53704,"Crying Game, The (1992)",Drama|Romance|War,False
312377,3261,1095,4,968251750,M,45,20,87505,Glengarry Glen Ross (1992),Drama,False


In [11]:
movielens_test.head()

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres,for_testing
693323,4653,2648,4,975532459,M,35,12,95051,Frankenstein (1931),Horror,False
24177,2259,1270,4,974591524,F,56,16,70503,Back to the Future (1985),Comedy|Sci-Fi,False
202202,3032,1378,5,970343147,M,25,0,47303,Young Guns (1988),Action|Comedy|Western,False
262003,3029,2289,4,972846393,M,18,4,92037,"Player, The (1992)",Comedy|Drama,False
777848,4186,2403,3,1017931262,M,25,7,33308,First Blood (1982),Action,False


# Mini-Challenge prep: evaluation functions

These are the two functions that you will need to test your `estimate` method.

In [12]:
def compute_rmse(y_pred, y_true):
    """ Compute Root Mean Squared Error. """
    
    return np.sqrt(np.mean(np.power(y_pred - y_true, 2)))

In [13]:
def evaluate(estimate_f):
    """ RMSE-based predictive performance evaluation with pandas. """
    
    ids_to_estimate = zip(movielens_test.user_id, movielens_test.movie_id)
    estimated = np.array([estimate_f(u,i) for (u,i) in ids_to_estimate])
    real = movielens_test.rating.values
    return compute_rmse(estimated, real)

Test a dummy solution!

In [14]:
def my_estimate_func(user_id, movie_id):
    return 3.0

You can test for performance with the following line, which assumes that your function is called `my_estimate_func`:

In [15]:
print ('RMSE for my estimate function: {}'.format(evaluate(my_estimate_func)))

RMSE for my estimate function: 1.2323719526527521


# Reco systems questions: Minimal reco engine v1.0

## 1. Simple collaborative filtering using mean ratings

In [16]:
# write an 'estimate' function that computes the mean rating of a particular user
def collab_mean(user_id, movie_id):
    # first, index into all ratings of this movie
    # second, compute the mean of those ratings
    user_condition = movielens_train.user_id != user_id
    movie_condition = movielens_train.movie_id == movie_id
    ratings_by_others = movielens_train.loc[movie_condition & user_condition]
    if ratings_by_others.empty:
        return 3.0
    else:
        return ratings_by_others.rating.mean()

print("RMSE for estimate: {}".format(evaluate(collab_mean)))
    
# try it out for a user_id, movie_id pair
collab_mean(4653, 2648)

RMSE for estimate: 1.1234279896011794


4.0

# Mini-Challenge: first round
Implement an `estimate` function of your own using other similarity notions, eg.:

- collaborative filter based on age similarities
- collaborative filter based on zip code similarities
- collaborative filter based on occupation similarities
- content filter based on movie genre

In [17]:
user_info = users.set_index('user_id')
user_info.head(5)

,gender,age,occupation,zip
user_id,,,,
1,F,1,10,48067
2,M,56,16,70072
3,M,25,15,55117
4,M,45,7,02460
5,M,25,20,55455


In [18]:
movie_info = movies.set_index('movie_id')
movie_info.head()

,title,genres
movie_id,,
1,Toy Story (1995),Animation|Children's|Comedy
2,Jumanji (1995),Adventure|Children's|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama
5,Father of the Bride Part II (1995),Comedy


In [19]:
def collab_age(user_id, movie_id):
    """ Collaborative filtering using an implicit sim(u,u') based on age. """
    
    user_condition = movielens_train.user_id != user_id
    movie_condition = movielens_train.movie_id == movie_id
    ratings_by_others = movielens_train.loc[user_condition & movie_condition]
    if ratings_by_others.empty: 
        return 3.0
    
    means = ratings_by_others.pivot_table('rating', index='movie_id', columns='age')
    user = user_info.ix[user_id, 'age']
    if user in means.columns: 
        return means.ix[movie_id, user]
    else:
        return means.ix[movie_id].mean()

print ('RMSE for collaborative filter based on age similarities: {}'.format(evaluate(collab_age)))

RMSE for collaborative filter based on age similarities: 1.2052013344107597


In [20]:
def collab_zip(user_id, movie_id):
    """ Collaborative filtering using an implicit sim(u,u') based on zip code. """
    
    user_condition = movielens_train.user_id != user_id
    movie_condition = movielens_train.movie_id == movie_id
    ratings_by_others = movielens_train.loc[user_condition & movie_condition]
    if ratings_by_others.empty: 
        return 3.0
    
    means = ratings_by_others.pivot_table('rating', index='movie_id', columns='zip')
    user = user_info.ix[user_id, 'zip']
    if user in means.columns: 
        return means.ix[movie_id, user]
    else:
        return means.ix[movie_id].mean()

print ('RMSE for collaborative filter based on zip similarities: {}'.format(evaluate(collab_zip)))

RMSE for collaborative filter based on zip similarities: 1.1256640319176328


In [21]:
def collab_occ(user_id, movie_id):
    """ Collaborative filtering using an implicit sim(u,u') based on occupation. """
    
    user_condition = movielens_train.user_id != user_id
    movie_condition = movielens_train.movie_id == movie_id
    ratings_by_others = movielens_train.loc[user_condition & movie_condition]
    if ratings_by_others.empty: 
        return 3.0
    
    means = ratings_by_others.pivot_table('rating', index='movie_id', columns='occupation')
    user = user_info.ix[user_id, 'occupation']
    if user in means.columns: 
        return means.ix[movie_id, user]
    else:
        return means.ix[movie_id].mean()

print ('RMSE for collaborative filter based on occupation similarities: {}'.format(evaluate(collab_occ)))

RMSE for collaborative filter based on occupation similarities: 1.202876964364191


# Mini-Challenge: second round
Implement an `estimate` function of your own using other custom similarity notions, eg.:

- euclidean
- cosine

In [22]:
overall_mean = movielens_train['rating'].mean()
overall_mean

3.5457348406988696

In [23]:
def my_estimate(user_id, movie_id):
    measures = [(0.4, movielens_train.loc[(movielens_train.movie_id == movie_id)]),
                (0.6, movielens_train.loc[(movielens_train.gender == users.loc[user_id].gender)])]
    
    return sum(weight * (measure.empty and overall_mean or measure.rating.mean()) 
              for weight, measure in measures)

print ('RMSE for my estimate: {}'.format(evaluate(my_estimate)))

RMSE for my estimate: 1.0483073970216743


In [24]:
def my_estimate(user_id, movie_id):
    measures = [(0.4, movielens_train.loc[(movielens_train.movie_id == movie_id)]),
                (0.5, movielens_train.loc[(movielens_train.occupation == users.loc[user_id].occupation)]),
                (0.05, movielens_train.loc[(movielens_train.gender == users.loc[user_id].gender)]),
                (0.05, movielens_train.loc[(movielens_train.zip == users.loc[user_id].zip)])]
    
    return sum(weight * (measure.empty and overall_mean or measure.rating.mean()) 
              for weight, measure in measures)

print ('RMSE for my estimate: {}'.format(evaluate(my_estimate)))

RMSE for my estimate: 1.047592760014394


In [25]:
def pearson(s1, s2):
    """Take two pd.Series objects and return a pearson correlation."""
    s1_c = s1 - s1.mean()
    s2_c = s2 - s2.mean()
    return np.sum(s1_c * s2_c) / np.sqrt(np.sum(s1_c ** 2) * np.sum(s2_c ** 2))

In [26]:
def euclidean(s1, s2):
    """Take two pd.Series objects and return their euclidean 'similarity'."""
    diff = s1 - s2
    return 1 / (1 + np.sqrt(np.sum(diff ** 2)))

In [27]:
def cosine(s1, s2):
    """Take two pd.Series objects and return their cosine similarity."""
    return np.sum(s1 * s2) / np.sqrt(np.sum(s1 ** 2) * np.sum(s2 ** 2))

In [28]:
class CollabEuclidean:
    """ Collaborative filtering using a custom sim(u,u'). """

    def learn(self):
        """ Prepare datastructures for estimation. """
        
        self.all_user_profiles = movielens_train.pivot_table('rating', index='movie_id', columns='user_id')

    def estimate(self, user_id, movie_id):
        """ Ratings weighted by similarity. """
        
        user_condition = movielens_train.user_id != user_id
        movie_condition = movielens_train.movie_id == movie_id
        ratings_by_others = movielens_train.loc[user_condition & movie_condition]
        if ratings_by_others.empty: 
            return 3.0
        
        ratings_by_others.set_index('user_id', inplace=True)
        their_ids = ratings_by_others.index
        their_ratings = ratings_by_others.rating
        their_profiles = self.all_user_profiles[their_ids]
        user_profile = self.all_user_profiles[user_id]
        sims = their_profiles.apply(lambda profile: euclidean(profile, user_profile), axis=0)
        ratings_sims = pd.DataFrame({'sim': sims, 'rating': their_ratings})
        ratings_sims = ratings_sims[ratings_sims.sim > 0]
        if ratings_sims.empty:
            return their_ratings.mean()
        else:
            return np.average(ratings_sims.rating, weights=ratings_sims.sim)
        
reco = CollabEuclidean()
reco.learn()
print ('RMSE for Collaborative Euclidean Recommender: {}'.format(evaluate(reco.estimate)))

RMSE for Collaborative Euclidean Recommender: 1.1234294760026475


In [29]:
class CollabCosine:
    """ Collaborative filtering using a custom sim(u,u'). """

    def learn(self):
        """ Prepare datastructures for estimation. """
        
        self.all_user_profiles = movielens_train.pivot_table('rating', index='movie_id', columns='user_id')

    def estimate(self, user_id, movie_id):
        """ Ratings weighted by similarity. """
        
        user_condition = movielens_train.user_id != user_id
        movie_condition = movielens_train.movie_id == movie_id
        ratings_by_others = movielens_train.loc[user_condition & movie_condition]
        if ratings_by_others.empty: 
            return 3.0
        
        ratings_by_others.set_index('user_id', inplace=True)
        their_ids = ratings_by_others.index
        their_ratings = ratings_by_others.rating
        their_profiles = self.all_user_profiles[their_ids]
        user_profile = self.all_user_profiles[user_id]
        sims = their_profiles.apply(lambda profile: cosine(profile, user_profile), axis=0)
        ratings_sims = pd.DataFrame({'sim': sims, 'rating': their_ratings})
        ratings_sims = ratings_sims[ratings_sims.sim > 0]
        if ratings_sims.empty:
            return their_ratings.mean()
        else:
            return np.average(ratings_sims.rating, weights=ratings_sims.sim)
        
reco = CollabCosine()
reco.learn()
print ('RMSE for Collaborative Consine Recommender: {}'.format(evaluate(reco.estimate)))

RMSE for Collaborative Consine Recommender: 1.1343992169551975
